In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### lenet

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras

from keras import models, layers

2024-05-07 16:24:21.107082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 16:24:21.107124: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 16:24:21.107908: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 16:24:21.168106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 16:24:22.218006: W tensorflow/compiler/

In [3]:
train_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-balanced-train.csv')
test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-balanced-test.csv')

In [4]:
print(train_data.shape)
print(test_data.shape)

(112799, 785)
(18799, 785)


In [5]:
y_train = train_data.iloc[:,:1]
x_train = train_data.iloc[:,1:]

y_test = test_data.iloc[:,:1]
x_test = test_data.iloc[:,1:]

In [6]:
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()

y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [7]:
x_train_np = np.reshape(x_train_np, newshape = (112799, 28, 28, 1))
x_test_np = np.reshape(x_test_np, newshape = (18799, 28, 28, 1))

y_train_np = tf.keras.utils.to_categorical(y_train_np,47)
y_test_np = tf.keras.utils.to_categorical(y_test_np,47)

In [8]:
model_lenet5 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5,
                    strides=1, activation='tanh',
                    input_shape=[28,28,1], padding='same'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=16, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=120, kernel_size=5,
                    strides=1, activation='tanh',padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(47, activation = "softmax")
])
model_lenet5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_lenet5.summary()

2024-05-03 18:33:00.707203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 18:33:00.740973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 18:33:00.741258: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 14, 14, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten (Flatten)           (None, 120)               0

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [10]:
model_lenet5.fit(x_train_np, y_train_np, batch_size=300, epochs = 50, callbacks=[callbacks])

Epoch 1/50


2024-05-03 18:33:07.213901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2024-05-03 18:33:09.201995: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1e400262e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-03 18:33:09.202023: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-05-03 18:33:09.208730: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 18:33:09.299730: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


376/376 [==============================] - 10s 7ms/step - loss: 1.0460 - accuracy: 0.7038
Epoch 2/50
376/376 [==============================] - 3s 8ms/step - loss: 0.4847 - accuracy: 0.8381
Epoch 3/50
376/376 [==============================] - 3s 8ms/step - loss: 0.4096 - accuracy: 0.8588
Epoch 4/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3751 - accuracy: 0.8683
Epoch 5/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3531 - accuracy: 0.8751
Epoch 6/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3366 - accuracy: 0.8791
Epoch 7/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3223 - accuracy: 0.8834
Epoch 8/50
376/376 [==============================] - 3s 7ms/step - loss: 0.3110 - accuracy: 0.8864
Epoch 9/50
376/376 [==============================] - 3s 8ms/step - loss: 0.3002 - accuracy: 0.8899
Epoch 10/50
376/376 [==============================] - 3s 9ms/step - loss: 0.2899 - accuracy: 0.8926
Epoch 11/

In [11]:
loss_lenet, acc_lenet = model_lenet5.evaluate(x_test_np,y_test_np)

588/588 [==============================] - 2s 3ms/step - loss: 0.6000 - accuracy: 0.8587


### Resnet

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

# 데이터 전처리
x_train_resized = tf.image.resize(x_train_np, [32, 32])  # ResNet에 맞게 크기 조정
x_test_resized = tf.image.resize(x_test_np, [32, 32])

# 이미지 채널을 3개로 조정 (RGB)
x_train_rgb = tf.repeat(x_train_resized, 3, axis=3)
x_test_rgb = tf.repeat(x_test_resized, 3, axis=3)

# ResNet-50 모델 정의
input_tensor = Input(shape=(32, 32, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output = Dense(47, activation='softmax')(x)
model_resnet = Model(inputs=base_model.input, outputs=output)

2024-05-07 16:24:32.381450: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 16:24:32.490870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 16:24:32.630144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# 모델 컴파일
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [15]:
# 모델 학습
model_resnet.fit(x_train_rgb, y_train_np, batch_size=1000, epochs=50, callbacks=[callbacks])

Epoch 1/50
113/113 [==============================] - 17s 119ms/step - loss: 0.1440 - accuracy: 0.9401
Epoch 2/50
113/113 [==============================] - 11s 96ms/step - loss: 0.1182 - accuracy: 0.9501
Epoch 3/50
113/113 [==============================] - 10s 93ms/step - loss: 0.1063 - accuracy: 0.9546
Epoch 4/50
113/113 [==============================] - 11s 96ms/step - loss: 0.0961 - accuracy: 0.9589
Epoch 5/50
113/113 [==============================] - 11s 96ms/step - loss: 0.0935 - accuracy: 0.9609
Epoch 6/50
113/113 [==============================] - 11s 94ms/step - loss: 0.0887 - accuracy: 0.9625
Epoch 7/50
113/113 [==============================] - 10s 87ms/step - loss: 0.0891 - accuracy: 0.9623
Epoch 8/50
113/113 [==============================] - 11s 94ms/step - loss: 0.0868 - accuracy: 0.9638
Epoch 9/50
113/113 [==============================] - 10s 91ms/step - loss: 0.0849 - accuracy: 0.9646
Epoch 10/50
113/113 [==============================] - 10s 92ms/step - loss: 0.08

In [16]:
loss_resnet, acc_resnet = model_resnet.evaluate(x_test_rgb,y_test_np)

588/588 [==============================] - 7s 10ms/step - loss: 0.7701 - accuracy: 0.8755


In [17]:
results = {"Model": ["lenet5", "resnet-50"], "Loss": [loss_lenet, loss_resnet], "Accuracy": [acc_lenet, acc_resnet]}
results_df = pd.DataFrame(results)

import IPython
display(results_df)

,Model,Loss,Accuracy
0,lenet5,0.600043,0.858663
1,resnet-50,0.770071,0.875525
